In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2 as ChiSquare

In [2]:
def meanVec(X):
    number_samples = X.shape[0]
    vec_one = np.ones((number_samples, 1))
    return 1/number_samples * X.T.dot(vec_one)

def covMat(X):
    number_samples = X.shape[0]
    vec_one = np.ones((number_samples, 1))
    I = np.eye(number_samples)
    return 1/(number_samples - 1) * X.T.dot(I - 1/number_samples * vec_one.dot(vec_one.T)).dot(X)

In [3]:
#def twoWayANOVA

In [4]:
def twoWayMANOVA(factorTable, significant_level = 0.01):
    g, b = factorTable.shape
    n, p = factorTable[0, 0].shape
    overallPop = np.vstack(factorTable.flatten().tolist())
    overallMean = meanVec(overallPop)
    SSP_fac1, SSP_fac2 = 0, 0
    SSP_int, SSP_res = 0, 0
    
    for i in range(g):
        factor_i = np.vstack(factorTable[i, :].flatten().tolist())
        mean_i = meanVec(factor_i)
        SSP_fac1 += b * n * (mean_i - overallMean).dot((mean_i - overallMean).T)
    
    for j in range(b):
        factor_j = np.vstack(factorTable[:, j].flatten().tolist())
        mean_j = meanVec(factor_j)
        SSP_fac2 += g * n * (mean_j - overallMean).dot((mean_j - overallMean).T)
    
    for i in range(b):
        for j in range(b):
            mean_i = meanVec(np.vstack(factorTable[i, :].flatten().tolist()))
            mean_j = meanVec(np.vstack(factorTable[:, j].flatten().tolist()))
            mean_ij = meanVec(factorTable[i, j])
            SSP_int += (
                n * (mean_ij - mean_i - mean_j + overallMean).dot((mean_ij - mean_i - mean_j + overallMean).T)
            )
            for r in range(n):
                sample = (factorTable[i, j][r, :]).reshape((p, 1))
                SSP_res += (sample - mean_ij).dot((sample - mean_ij).T)
    print('>> SSP_fac1 = ')
    print(SSP_fac1)
    
    print('>> SSP_fac2 = ')
    print(SSP_fac2)
    
    print('>> SSP_int = ')
    print(SSP_int)
    
    print('>> SSP_res = ')
    print(SSP_res)
    
    # interaction
    wilks_lambda = np.linalg.det(SSP_res)/np.linalg.det(SSP_int + SSP_res)
    statistic = -(g*b*(n - 1) - (p + 1 - (g - 1)*(b - 1))/2) * np.log(wilks_lambda)
    critical_value = ChiSquare.ppf(1 - significant_level, df = (g - 1)*(b - 1)* p)
    p_value = 1 - ChiSquare.cdf(statistic, df = (g - 1) * (b - 1) * p)
    print('>> Interaction effects test:')
    print(f">> Wilk's lambda = {wilks_lambda}")
    print(f'>> Statistic = {statistic}')
    print(f'>> Critical value = {critical_value}')
    print(f'>> p-value = {p_value}')
    print(f'>> Significant level = {significant_level}')
    
    if statistic > critical_value:
        print(f'>> Conclusion: Reject H_0')
    else:
        print(f">> Conclusion: Accept H_0")
    print('-' * 50)
    
    # factor 1
    wilks_lambda = np.linalg.det(SSP_res)/np.linalg.det(SSP_fac1 + SSP_res)
    statistic = -(g*b*(n - 1) - (p + 1 - (g - 1))/2) * np.log(wilks_lambda)
    critical_value = ChiSquare.ppf(1 - significant_level, df = (g - 1)*p)
    p_value = 1 - ChiSquare.cdf(statistic, df = (g - 1)*p)
    print('>> Factor 1 effects test:')
    print(f">> Wilk's lambda = {wilks_lambda}")
    print(f'>> Statistic = {statistic}')
    print(f'>> Critical value = {critical_value}')
    print(f'>> p-value = {p_value}')
    print(f'>> Significant level = {significant_level}')
    
    if statistic > critical_value:
        print(f'>> Conclusion: Reject H_0')
    else:
        print(f'>> Conclusion: Accept H_0')
    print('-' * 50)

    # factor 2
    wilks_lambda = np.linalg.det(SSP_res)/np.linalg.det(SSP_fac2 + SSP_res)
    statistic = -(g*b*(n - 1) - (p + 1 - (b - 1))/2) * np.log(wilks_lambda)
    critical_value = ChiSquare.ppf(1 - significant_level, df = (b - 1)* p)
    p_value = 1 - ChiSquare.cdf(statistic, df = (b - 1) * p)
    print('>> Factor 2 effects test:')
    print(f">> Wilk's lambda = {wilks_lambda}")
    print(f'>> Statistic = {statistic}')
    print(f'>> Critical value = {critical_value}')
    print(f'>> p-value = {p_value}')
    print(f'>> Significant level = {significant_level}')
    
    if statistic > critical_value:
        print(f'>> Conclusion: Reject H_0')
    else:
        print(f'>> Conclusion: Accept H_0')
    print('-' * 50)

In [5]:
data = pd.read_csv('T6-18.csv', delimiter = '  ', engine = 'python')

In [6]:
data.head()

,X560nm,X720nm,Species,Time,Replication
0,9.33,19.14,SS,1,1
1,8.74,19.55,SS,1,2
2,9.31,19.24,SS,1,3
3,8.27,16.37,SS,1,4
4,10.22,25.00,SS,2,1


In [7]:
factorTable = np.zeros((3, 3), dtype = np.object)
for row_index, species in enumerate(data['Species'].unique()):
    for col_index, time in enumerate(data['Time'].unique()):
        factorTable[row_index, col_index] = (
            data.loc[(data['Species'] == species) & (data['Time'] == time)]
            .iloc[:, :2].to_numpy()
        )

In [8]:
twoWayMANOVA(factorTable, significant_level = 0.05)

>> SSP_fac1 = 
[[ 965.18117222 1377.60191389]
 [1377.60191389 2026.85637222]]
>> SSP_fac2 = 
[[1275.24773889 2644.92736389]
 [2644.92736389 5573.80570556]]
>> SSP_int = 
[[795.80794444 375.96311944]
 [375.96311944 193.54926111]]
>> SSP_res = 
[[  76.658775   37.9299  ]
 [  37.9299   1769.642225]]
>> Interaction effects test:
>> Wilk's lambda = 0.08707032115867996
>> Statistic = 67.12857793502033
>> Critical value = 15.50731305586545
>> p-value = 1.8283374814132003e-11
>> Significant level = 0.05
>> Conclusion: Reject H_0
--------------------------------------------------
>> Factor 1 effects test:
>> Wilk's lambda = 0.06877382340376705
>> Statistic = 70.93870012593094
>> Critical value = 9.487729036781154
>> p-value = 1.4432899320127035e-14
>> Significant level = 0.05
>> Conclusion: Reject H_0
--------------------------------------------------
>> Factor 2 effects test:
>> Wilk's lambda = 0.049166033502360394
>> Statistic = 79.83263515159176
>> Critical value = 9.487729036781154
>> p-val

## c)

In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [10]:
model = ols('X560nm ~ C(Species) + C(Time) + C(Species):C(Time)', data = data).fit()
sm.stats.anova_lm(model, test = 'F', typ = 2)

,sum_sq,df,F,PR(>F)
C(Species),965.181172,2.0,169.973311,5.026570e-16
C(Time),1275.247739,2.0,224.577610,1.492126e-17
C(Species):C(Time),795.807944,4.0,70.072912,7.341395e-14
Residual,76.658775,27.0,NaN,NaN


In [11]:
model = ols('X720nm ~ C(Species) + C(Time) + C(Species):C(Time)', data = data).fit()
sm.stats.anova_lm(model, test = 'F', typ = 2)

,sum_sq,df,F,PR(>F)
C(Species),2026.856372,2.0,15.462199,3.347959e-05
C(Time),5573.805706,2.0,42.520672,4.537364e-09
C(Species):C(Time),193.549261,4.0,0.738261,5.741051e-01
Residual,1769.642225,27.0,NaN,NaN
